In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import segmentation_models_pytorch as smp
from dataloader import get_dataloader

c:\Users\ILY\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# setting

In [9]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)
BATCH_SIZE = 16
EPOCHS = 20
LR = 1e-4

images_dir = "Dataset700_label/images"
masks_dir = "Dataset700_label/predicted_masks"

train_loader = get_dataloader(images_dir, masks_dir, batch_size=BATCH_SIZE, mode='train')
val_loader = get_dataloader(images_dir, masks_dir, batch_size=BATCH_SIZE, mode='val')


cpu


# deeplab3 + resten50

In [10]:
model_dl3 = smp.DeepLabV3(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,  
).to(DEVICE)

In [11]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for images, masks in loader:
        images = images.to(DEVICE)
        masks = masks.to(DEVICE).unsqueeze(1)

        preds = model(images)
        loss = criterion(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(loader)


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_dl3.parameters(), lr=LR)

for epoch in range(EPOCHS):
    loss = train_one_epoch(model_dl3, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f}")


Epoch 1/20 - Loss: 0.5041
Epoch 2/20 - Loss: 0.4118
